# Exploring Hacker News Posts
-----

For this project, we will be working with a data set of user-submitted posts that are akin to reddit in that they are voted and commented upon on the technology site [Hacker News](https://news.ycombinator.com/) .

Specifically, we'll be looking at two types of posts:
- __Ask HN:__ posts that ask the Hacker News community a specific questions
- __Show HN:__ posts that aim to present the community a project, product, or general items of interest

We'll compare these two types of posts to determine the following:
- Do __Ask HN__ or __Show HN__ receive more comments on average?
- Do posts created at a certain time receive more comments on average?

The data set can be found [here](https://www.kaggle.com/hacker-news/hacker-news-posts). Note that this has been reduced from from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:
- __id__: The unique identifier from Hacker News for the post
- __title__: The title of the post
- __url__: The URL that the posts links to, if it the post has a URL
- __num_points__: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- __num_comments__: The number of comments that were made on the post
- __author__: The username of the person who submitted the post
- __created_at__: The date and time at which the post was submitted

In the below cell node, we open and read the CSV file and print the first few rows.

In [2]:
from csv import reader

hn = list(reader(open("hacker_news.csv")))

for row in hn[0:6]:
    print(row)
    print("\n")

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




Since the above output is a bit hard to read, let's look at the table version of the above.

|    id    |                             title                            |                                               url                                              | num_points | num_comments |   author   |    created_at   |
|:--------:|:------------------------------------------------------------:|:----------------------------------------------------------------------------------------------:|:----------:|:------------:|:----------:|:---------------:|
| 12224879 | Interactive Dynamic Video                                    | http://www.interactivedynamicvideo.com/                                                        | 386        | 52           | neophyte   | 8/4/2016 11:52  |
| 10975351 | How to Use Open Source and Shut the Fuck Up at the Same Time | http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/ | 39         | 10           | josep2     | 1/26/2016 19:30 |
| 11964716 | Florida DJs May Face Felony for April Fools' Water Joke      | http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/          | 2          | 1            | vezycash   | 6/23/2016 22:20 |
| 11919867 | Technology ventures: From Idea to Enterprise                 | https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429               | 3          | 1            | hswarna    | 6/17/2016 0:01  |
| 10301696 | Note by Note: The Making of Steinway L1037 (2007)            | http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0                                     | 8          | 2            | walterbell | 9/30/2015 4:12  |

### Data Preparation

Before we proceed with the analysis, we first prepare the data by removing the header row into a separate list.

In [3]:
headers = hn[0]

hn = hn[1:]

print("========================")
print("HEADER ROW")
print("========================")
print(headers)
print("\n")
print("========================")
print("FIRST FIVE")
print("========================")
for row in hn[0:5]: # Confirm if header is removed
    print(row)
    print("\n")

HEADER ROW
['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


FIRST FIVE
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/20

Additionally, since we're only looking at the two aforementioned types of posts, we filter the dataset by looking at the post titles. 

To find the posts that begin with either _"Ask HN"_ or _"Show HN"_, we'll use the string method ___startswith___. It's important to note that case will matter when matching using this method. If we wish to control for case, we can use the ___lower___ method which returns a lowercase version of the starting string.

In [4]:
ask_posts   = []
show_posts  = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith("ask hn"):
        ask_posts.append(row)
    elif title.startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)

print("========================")
print("ASK HN POSTS")
print("========================")
print("# of posts: ", len(ask_posts))
print(ask_posts[0:3])
print("\n")
print("========================")
print("SHOW HN POSTS")
print("========================")
print("# of posts: ", len(show_posts))
print(show_posts[0:3])
print("\n")
print("========================")
print("OTHER POSTS")
print("========================")
print("# of posts: ", len(other_posts))
print(other_posts[0:3])

ASK HN POSTS
# of posts:  1744
[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14']]


SHOW HN POSTS
# of posts:  1162
[['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', '1', 'h8liu', '4/28/2016 18:05']]


OTHER POSTS
# of posts:  17194
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/20

There seems to be more __Ask HN__ posts than there are __Show HN__ posts (1744 vs. 1162).

### Do Ask HN or Show HN receive more comments on average?
Now let's find out which of the two garner more comments on the average. As we can see in the output below, __Ask HN__ posts receive more comments on average than __Show HN__ posts. This seems logical as the former aims specifically to get inputs and posts from the community to answer the original poster's question/s.

In [5]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
    
ave_ask_comments = total_ask_comments/len(ask_posts)
print("Average # of Comments on Ask Posts:", round(ave_ask_comments))

total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    
ave_show_comments = total_show_comments/len(show_posts)
print("Average # of Comments on Show Posts:", round(ave_show_comments))


Average # of Comments on Ask Posts: 14
Average # of Comments on Show Posts: 10


### Do posts created at a certain time receive more comments on average?

For this part, we will try to find out if there are certain time periods that attract more comments on the average for __Ask HN__ posts. To do this, we perform the following:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

We'll make use of the ___datetime___ module as well as the ___datetime.strptime()___ constructor to get datetime objects since the column in the dataset we are interested in stores the dates as strings.

We also create two dictionaries to get two frequencies:
- __counts_by_hour:__ contains the number of ask posts created during each hour of the day.
- __comments_by_hour:__ contains the corresponding number of comments ask posts created at each hour received.

In [11]:
import datetime as dt

# Getting datetime and number of comments in the posts
result_list = []

for row in ask_posts:
    created_at   = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])
    
print(result_list[0:3])
print("\n")

# Creating Hour Dictionarys
counts_by_hour   = {}
comments_by_hour = {}

for row in result_list:
    hour = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    hour = hour.hour
    if hour in counts_by_hour:
        counts_by_hour[hour]   += 1
        comments_by_hour[hour] += int(row[1])
    else:
        counts_by_hour[hour]   = 1
        comments_by_hour[hour] = int(row[1])
        
print(counts_by_hour)
print("\n")
print(comments_by_hour)

[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1]]


{0: 55, 1: 60, 2: 58, 3: 54, 4: 47, 5: 46, 6: 44, 7: 34, 8: 48, 9: 45, 10: 59, 11: 58, 12: 73, 13: 85, 14: 107, 15: 116, 16: 108, 17: 100, 18: 109, 19: 110, 20: 80, 21: 109, 22: 71, 23: 68}


{0: 447, 1: 683, 2: 1381, 3: 421, 4: 337, 5: 464, 6: 397, 7: 267, 8: 492, 9: 251, 10: 793, 11: 641, 12: 687, 13: 1253, 14: 1416, 15: 4477, 16: 1814, 17: 1146, 18: 1439, 19: 1188, 20: 1722, 21: 1745, 22: 479, 23: 543}


The above __result_list__ contains the date and time the post was created along with the number of comments it was able to get.

Upon inspection, the hour with the most number of ask posts is __15:00__ or __3:00 PM__, while the hour with the most number of comments is, unsurprisingly, that as well.

Using the frequencies in these dictionaries, we'll compute the average number of comments per post for each hour below.

In [10]:
avg_by_hour = []

for hour in comments_by_hour:
    hourlist = [hour, round(comments_by_hour[hour]/counts_by_hour[hour])]
    avg_by_hour.append(hourlist)
    print("Hour:", hourlist[0], "=>", hourlist[1], "ave. # of comments per post")

Hour: 0 => 8 ave. # of comments per post
Hour: 1 => 11 ave. # of comments per post
Hour: 2 => 24 ave. # of comments per post
Hour: 3 => 8 ave. # of comments per post
Hour: 4 => 7 ave. # of comments per post
Hour: 5 => 10 ave. # of comments per post
Hour: 6 => 9 ave. # of comments per post
Hour: 7 => 8 ave. # of comments per post
Hour: 8 => 10 ave. # of comments per post
Hour: 9 => 6 ave. # of comments per post
Hour: 10 => 13 ave. # of comments per post
Hour: 11 => 11 ave. # of comments per post
Hour: 12 => 9 ave. # of comments per post
Hour: 13 => 15 ave. # of comments per post
Hour: 14 => 13 ave. # of comments per post
Hour: 15 => 39 ave. # of comments per post
Hour: 16 => 17 ave. # of comments per post
Hour: 17 => 11 ave. # of comments per post
Hour: 18 => 13 ave. # of comments per post
Hour: 19 => 11 ave. # of comments per post
Hour: 20 => 22 ave. # of comments per post
Hour: 21 => 16 ave. # of comments per post
Hour: 22 => 7 ave. # of comments per post
Hour: 23 => 8 ave. # of comme

We now have the computed averages! In order to identify the hours with the most number of comments on the average, we sort this list and print the five sorted values below.

A list swapping the position of columns was first created, and from there we make use of the ___sorted()___ function. Furthermore, we make use of the ___strftime()___ function to format our datetime object.

In [12]:
swap_avg_by_hour = []

for hour in avg_by_hour:
    swap_avg_by_hour.append([hour[1], hour[0]])
    
print(swap_avg_by_hour)
print("\n")

sorted_swap = sorted(swap_avg_by_hour, reverse = True)
print(sorted_swap)
print("\n")

print("Top Hours for Ask Posts Comments")
for row in sorted_swap:
    template = "{hour}: {ave} average comments per post"
    hour = str(row[1])
    hour = dt.datetime.strptime(hour, "%H")
    hour = hour.strftime("%H:%M")
    print(template.format(hour = hour, ave = row[0]))

[[8, 0], [11, 1], [24, 2], [8, 3], [7, 4], [10, 5], [9, 6], [8, 7], [10, 8], [6, 9], [13, 10], [11, 11], [9, 12], [15, 13], [13, 14], [39, 15], [17, 16], [11, 17], [13, 18], [11, 19], [22, 20], [16, 21], [7, 22], [8, 23]]


[[39, 15], [24, 2], [22, 20], [17, 16], [16, 21], [15, 13], [13, 18], [13, 14], [13, 10], [11, 19], [11, 17], [11, 11], [11, 1], [10, 8], [10, 5], [9, 12], [9, 6], [8, 23], [8, 7], [8, 3], [8, 0], [7, 22], [7, 4], [6, 9]]


Top Hours for Ask Posts Comments
15:00: 39 average comments per post
02:00: 24 average comments per post
20:00: 22 average comments per post
16:00: 17 average comments per post
21:00: 16 average comments per post
13:00: 15 average comments per post
18:00: 13 average comments per post
14:00: 13 average comments per post
10:00: 13 average comments per post
19:00: 11 average comments per post
17:00: 11 average comments per post
11:00: 11 average comments per post
01:00: 11 average comments per post
08:00: 10 average comments per post
05:00: 10 avera

### Conclusion
____

Thus, to get better chances of getting comments, it's best to create a post at __3:00 PM__. In general, there seems to be no apparent pattern in the average number of comments received by posts by the hour it was posted, with the top hours being from the afternoon to the late evening with __2:00 AM__ as an outlier.